In [ ]:
#importing the Libraies
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
dataset=pd.read_csv("/content/drive/MyDrive/AI Project/CKD.csv")


In [ ]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [ ]:
dataset=pd.get_dummies(dataset,drop_first=True)

In [ ]:
dataset

,age,bp,al,su,bgr,bu,sc,sod,pot,hrmo,...,pc_normal,pcc_present,ba_present,htn_yes,dm_yes,cad_yes,appet_yes,pe_yes,ane_yes,classification_yes
0,2.000000,76.459948,3.0,0.0,148.112676,57.482105,3.077356,137.528754,4.627244,12.518156,...,0,0,0,0,0,0,1,1,0,1
1,3.000000,76.459948,2.0,0.0,148.112676,22.000000,0.700000,137.528754,4.627244,10.700000,...,1,0,0,0,0,0,1,0,0,1
2,4.000000,76.459948,1.0,0.0,99.000000,23.000000,0.600000,138.000000,4.400000,12.000000,...,1,0,0,0,0,0,1,0,0,1
3,5.000000,76.459948,1.0,0.0,148.112676,16.000000,0.700000,138.000000,3.200000,8.100000,...,1,0,0,0,0,0,1,0,1,1
4,5.000000,50.000000,0.0,0.0,148.112676,25.000000,0.600000,137.528754,4.627244,11.800000,...,1,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,51.492308,70.000000,0.0,0.0,219.000000,36.000000,1.300000,139.000000,3.700000,12.500000,...,1,0,0,0,0,0,1,0,0,1
395,51.492308,70.000000,0.0,2.0,220.000000,68.000000,2.800000,137.528754,4.627244,8.700000,...,1,0,0,1,1,0,1,0,1,1
396,51.492308,70.000000,3.0,0.0,110.000000,115.000000,6.000000,134.000000,2.700000,9.100000,...,1,0,0,1,1,0,0,0,0,1
397,51.492308,90.000000,0.0,0.0,207.000000,80.000000,6.800000,142.000000,5.500000,8.500000,...,1,0,0,1,1,0,1,0,1,1


In [ ]:
indep=dataset.drop(['classification_yes'],axis=1)
dep=dataset['classification_yes']

In [ ]:
#split into training set and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(indep, dep, test_size = 1/3, random_state = 0)


In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
from sklearn.svm import SVC

In [ ]:
#https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {'kernel':['linear','rbf','poly','sigmoid'],
             'gamma':['auto','scale'],
             'C':[10,100,1000,2000,3000]}



grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1_weighted')

# fitting the model for grid search
grid.fit(X_train, y_train)




Fitting 5 folds for each of 40 candidates, totalling 200 fits


GridSearchCV(estimator=SVC(), n_jobs=-1,
             param_grid={'C': [10, 100, 1000, 2000, 3000],
                         'gamma': ['auto', 'scale'],
                         'kernel': ['linear', 'rbf', 'poly', 'sigmoid']},
             scoring='f1_weighted', verbose=3)

In [ ]:
# print best parameter after tuning
#print(grid.best_params_)
re=grid.cv_results_
#print(re)
grid_predictions = grid.predict(X_test)


from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions)



# print classification report
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)




In [ ]:

from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)


The f1_macro value for best parameter {'C': 10, 'gamma': 'auto', 'kernel': 'sigmoid'}: 0.9924946382275899


In [ ]:
print("The confusion Matrix:\n",cm)

The confusion Matrix:
 [[51  0]
 [ 1 81]]


In [ ]:
print("The report:\n",clf_report)

The report:
               precision    recall  f1-score   support

           0       0.98      1.00      0.99        51
           1       1.00      0.99      0.99        82

    accuracy                           0.99       133
   macro avg       0.99      0.99      0.99       133
weighted avg       0.99      0.99      0.99       133



In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

# Assuming you have already trained and fitted the LogisticRegression model
logistic_regression_model = LogisticRegression()
logistic_regression_model.fit(X_train, y_train)

# Calculate the ROC AUC score using predict_proba for the positive class (class index 1)
roc_auc = roc_auc_score(y_test, logistic_regression_model.predict_proba(X_test)[:, 1])
print("ROC AUC Score: {:.4f}".format(roc_auc))


ROC AUC Score: 1.0000


In [ ]:
table=pd.DataFrame.from_dict(re)

In [ ]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_gamma,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.006541,0.002683,0.004408,0.002362,10,auto,linear,"{'C': 10, 'gamma': 'auto', 'kernel': 'linear'}",0.981569,0.962636,0.962573,0.981031,0.981217,0.973805,0.009147,23
1,0.006219,0.002650,0.006556,0.003555,10,auto,rbf,"{'C': 10, 'gamma': 'auto', 'kernel': 'rbf'}",0.963284,0.981014,1.000000,1.000000,1.000000,0.988859,0.014751,4
2,0.005426,0.002275,0.009825,0.005878,10,auto,poly,"{'C': 10, 'gamma': 'auto', 'kernel': 'poly'}",1.000000,0.981014,1.000000,1.000000,0.981217,0.992446,0.009252,3
3,0.006952,0.002600,0.006952,0.003316,10,auto,sigmoid,"{'C': 10, 'gamma': 'auto', 'kernel': 'sigmoid'}",0.981569,1.000000,1.000000,0.981031,1.000000,0.992520,0.009163,1
4,0.005598,0.003526,0.007146,0.003485,10,scale,linear,"{'C': 10, 'gamma': 'scale', 'kernel': 'linear'}",0.981569,0.962636,0.962573,0.981031,0.981217,0.973805,0.009147,23
5,0.002917,0.000092,0.004195,0.001470,10,scale,rbf,"{'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}",0.963284,0.981014,1.000000,1.000000,1.000000,0.988859,0.014751,4
6,0.006339,0.004026,0.010093,0.001042,10,scale,poly,"{'C': 10, 'gamma': 'scale', 'kernel': 'poly'}",1.000000,0.981014,1.000000,0.981031,0.981217,0.988652,0.009266,6
7,0.002973,0.000209,0.008153,0.003479,10,scale,sigmoid,"{'C': 10, 'gamma': 'scale', 'kernel': 'sigmoid'}",0.981569,1.000000,1.000000,0.981031,1.000000,0.992520,0.009163,1
8,0.005708,0.003592,0.006447,0.004695,100,auto,linear,"{'C': 100, 'gamma': 'auto', 'kernel': 'linear'}",0.981569,0.962636,0.962573,0.981031,0.981217,0.973805,0.009147,23
9,0.005851,0.003155,0.004364,0.002795,100,auto,rbf,"{'C': 100, 'gamma': 'auto', 'kernel': 'rbf'}",0.963284,0.981014,1.000000,0.981031,1.000000,0.985066,0.013807,9


In [ ]:
age_input=float(input("Age:"))
EstimatedSalary_input=float(input("EstimatedSalary':"))
Gender_Male_input=int(input("Sex Male 0 or 1:"))


In [ ]:
Future_Prediction=grid.predict([[age_input,EstimatedSalary_input,Gender_Male_input  ]])# change the paramter,play with it.
print("Future_Prediction={}".format(Future_Prediction))

In [ ]:
f = open("/content/drive/MyDrive/AI Project/myfile.txt", "x")

In [ ]:
f = open("/content/drive/MyDrive/AI Project/myfile.txt", "x")

In [ ]:
f.writelines(table)

In [ ]:
print(f.readlines)